In [7]:
import sys
sys.path.append("../src")

import generate_general_networks as generate
import torch
import matplotlib.pyplot as plt
import json
import numpy as np
import tikzplotlib
import loading_json as load
import json

In [8]:
# Load results from optimization
f = open("../optimization_results/general_optimization/single_junction_1.json")
results = json.load(f)
f.close()

In [13]:
# Collecting network configuration
network_file = results['network_file']
f = open("../"+network_file)
network_config = json.load(f)
f.close()

T = network_config['T']
N = network_config['N']
controls = network_config['control_points']

In [10]:
# Collecting the start and final parameters
start = results['parameters'][0]
opt = results['parameters'][-1]

In [14]:
# Create the networks
start_speed = [[torch.tensor(start[i])] for i in range(2)]
opt_speed = [[torch.tensor(opt[i])] for i in range(2)]
start_cycle = [torch.tensor(start[2]), torch.tensor(start[3])]
opt_cycle = [torch.tensor(opt[2]), torch.tensor(opt[3])]

start_network = generate.single_junction_network(T, N, start_speed, controls, start_cycle, track_grad=False)
opt_network = generate.single_junction_network(T, N, opt_speed, controls, opt_cycle, track_grad=False)

In [15]:
# Do the simulations
densities, _, lengths, delays, n_stops = start_network.solve_cons_law_counting()
opt_densities, _, opt_lengths, opt_delays, opt_n_stops = opt_network.solve_cons_law_counting()

Bus bus_1 reached bus stop 0 at time 113.23703002929688, should wait for 29.854145050048828 seconds
Bus bus_1 reached bus stop 0 at time 34.9142951965332, should wait for 29.798316955566406 seconds


In [16]:
# Store densities
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(lengths)

opt_densities_loaded = load.convert_from_tensor(opt_densities)
opt_bus_lengths_loaded = load.convert_from_tensor(opt_lengths)

In [17]:
with open("../general_densities/single_junction_start.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [18]:
with open("../general_densities/single_junction_optimal.json", 'w') as fd:
    fd.write(json.dumps([opt_densities_loaded, opt_bus_lengths_loaded]))

In [19]:
# Convert to the same time scale:
# Use single_lane_optimal as the goal


f = open("../general_densities/single_junction_start.json")
data = json.load(f)
f.close()
orig_densities = data[0]
orig_lengths = data[1]

f = open("../general_densities/single_junction_optimal.json")
data_goal = json.load(f)
f.close()
goal_densities = data_goal[0]

In [20]:
goal_times = list(goal_densities['0'].keys())
goal_times = [float(t) for t in goal_times]

old_times = list(orig_densities['0'].keys())
old_times = [float(t) for t in old_times]

In [21]:
new_lengths = {i : {} for i in orig_lengths.keys()}
new_densities = {i : {} for i in orig_densities.keys()}

prev_idx = 0
for t in goal_times:
    add_incr = 0
    for t_prev, t_next in zip(old_times[prev_idx:-1], old_times[prev_idx+1:]):
        if t_prev <= t and t_next >= t:
            prev_dist = t - t_prev
            next_dist = t_next - t
            interval_length = t_next - t_prev
            prev_weight = (interval_length - prev_dist) / interval_length
            next_weight = (interval_length - next_dist) / interval_length
            for bus_id in orig_lengths.keys(): 
                new_lengths[bus_id][str(t)] = orig_lengths[bus_id][str(t_prev)]*prev_weight + orig_lengths[bus_id][str(t_next)]*next_weight
            for road_id in orig_densities.keys():
                new_densities[road_id][str(t)] = [orig_densities[road_id][str(t_prev)][i]*prev_weight + orig_densities[road_id][str(t_next)][i]*next_weight for i in range(len(orig_densities[road_id][str(t_prev)]))]
            break
        else:
            add_incr += 1

In [22]:
# Write to file
with open("../general_densities/single_junction_start_opt_times.json", 'w') as fd:
    fd.write(json.dumps([new_densities, new_lengths]))